In [1]:
# for gioia's python build
import sys
sys.path.append('/Users/maggiori/anaconda/envs/py35/lib/python3.5/site-packages')

In [2]:
import numpy as np

In [3]:
# to import modules locally without having installed the entire package
# http://stackoverflow.com/questions/714063/importing-modules-from-parent-folder
import os, sys, inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)

In [4]:
from tsdb import PersistentDB
from tsdb import BitMapIndex
from timeseries import TimeSeries

In [5]:
# synthetic data
t = np.array([1, 1.5, 2, 2.5, 10, 11, 12])
v1 = np.array([10, 12, -11, 1.5, 10, 13, 17])
v2 = np.array([8, 12, -11, 1.5, 10, 13, 17])
a1 = TimeSeries(t, v1)
a2 = TimeSeries(t, v2)

In [6]:
identity = lambda x: x

# index: 1 is binary tree index, 2 is bitmap index
schema = {
  'pk':         {'type': 'str', 'convert': identity,   'index': None, 'values': None},
  'ts':         {'type': 'int', 'convert': identity,   'index': None, 'values': None},
  'order':      {'type': 'int', 'convert': int,        'index': 1, 'values': None},
  'blarg':      {'type': 'int', 'convert': int,        'index': 1, 'values': None},
  'useless':    {'type': 'int', 'convert': identity,   'index': 1, 'values': None},
  'mean':       {'type': 'float', 'convert': float,    'index': 1, 'values': None},
  'std':        {'type': 'float', 'convert': float,    'index': 1, 'values': None},
  'vp':         {'type': 'bool', 'convert': bool,      'index': 2, 'values': [True, False]},
  'deleted':    {'type': 'bool', 'convert': bool,      'index': 2, 'values': [True, False]}
}

In [7]:
# Delete any default db present (otherwise the db creation will load
# the previous one...)
filelist = ["../db_files/default/" + f for f in os.listdir("../db_files/default/")]
for f in filelist:
    os.remove(f)

In [8]:
# create persistent db
ddb = PersistentDB(schema, 'pk', len(t), data_dir='../db_files')

In [9]:
ddb.insert_ts('pk1', a1)

In [10]:
ddb.upsert_meta('pk1', {'order': 1, 'blarg': 2})

In [11]:
ddb.insert_ts('pk2', a2)

In [12]:
ddb.upsert_meta('pk2', {'order': 2, 'blarg': 2})

In [13]:
ddb.indexes

{'blarg': <tsdb.indexes.BinTreeIndex at 0x107465da0>,
 'deleted': <tsdb.indexes.BitMapIndex at 0x107465cc0>,
 'mean': <tsdb.indexes.BinTreeIndex at 0x107465e48>,
 'order': <tsdb.indexes.BinTreeIndex at 0x107465d30>,
 'std': <tsdb.indexes.BinTreeIndex at 0x107465be0>,
 'useless': <tsdb.indexes.BinTreeIndex at 0x107465f60>,
 'vp': <tsdb.indexes.BitMapIndex at 0x107465860>}

In [14]:
# print metadata
for field, index in ddb.indexes.items():
    print(field, index.items())

useless [(0, {'pk2', 'pk1'})]
blarg [(2, {'pk2', 'pk1'})]
deleted [(False, {'pk2', 'pk1'})]
vp [(False, {'pk2', 'pk1'})]
std [(0.0, {'pk2', 'pk1'})]
mean [(0.0, {'pk2', 'pk1'})]
order [(1, {'pk1'}), (2, {'pk2'})]


In [18]:
print(a1.times())
print(a1.values())
print(a2.times())
print(a2.values())

[  1.    1.5   2.    2.5  10.   11.   12. ]
[ 10.   12.  -11.    1.5  10.   13.   17. ]
[  1.    1.5   2.    2.5  10.   11.   12. ]
[  8.   12.  -11.    1.5  10.   13.   17. ]


In [19]:
print(ddb._get_ts('pk1').times())
print(ddb._get_ts('pk1').values())
print(ddb._get_ts('pk2').times())
print(ddb._get_ts('pk2').values())

[  1.    1.5   2.    2.5  10.   11.   12. ]
[ 10.   12.  -11.    1.5  10.   13.   17. ]
[  1.    1.5   2.    2.5  10.   11.   12. ]
[  8.   12.  -11.    1.5  10.   13.   17. ]


In [15]:
ddb.pks.keys()

dict_keys(['pk1', 'pk2'])

In [16]:
ddb.pks.keys()

dict_keys(['pk1', 'pk2'])

In [17]:
ddb.select({'pk': 'pk1'}, [], None)

(['pk1'],
 [{'blarg': 2,
   'mean': 0.0,
   'order': 1,
   'pk': 'pk1',
   'std': 0.0,
   'useless': 0,
   'vp': False}])

In [18]:
ddb.select({'pk': 'pk2'}, [], None)

(['pk2'],
 [{'blarg': 2,
   'mean': 0.0,
   'order': 2,
   'pk': 'pk2',
   'std': 0.0,
   'useless': 0,
   'vp': False}])

In [19]:
ddb.delete_ts('pk1')

In [20]:
ddb.insert_ts('pk1', a1)

In [21]:
# print metadata
for field, index in ddb.indexes.items():
    if isinstance(index, BitMapIndex):
        print(field)
        for k in index.keys():
            print(k, index[k])
        print('-----')

vp
False {'pk1', 'pk2'}
-----
deleted
False {'pk1', 'pk2'}
-----


In [22]:
ddb.insert_ts('pk3', a1)
ddb.insert_ts('pk4', a1)
ddb.insert_ts('pk5', a1)
ddb.insert_ts('pk6', a1)
ddb.insert_ts('pk7', a1)

In [23]:
# print metadata
for field, index in ddb.indexes.items():
    print(field)
    for k in index.keys():
        print(k, index[k])
    print('-----')

std
0.0 {'pk4', 'pk7', 'pk6', 'pk3', 'pk1', 'pk5', 'pk2'}
-----
blarg
0 {'pk4', 'pk7', 'pk6', 'pk3', 'pk1', 'pk5'}
2 {'pk2'}
-----
vp
False {'pk4', 'pk7', 'pk6', 'pk3', 'pk1', 'pk5', 'pk2'}
-----
deleted
False {'pk4', 'pk7', 'pk6', 'pk3', 'pk1', 'pk5', 'pk2'}
-----
mean
0.0 {'pk4', 'pk7', 'pk6', 'pk3', 'pk1', 'pk5', 'pk2'}
-----
order
0 {'pk4', 'pk7', 'pk6', 'pk3', 'pk1', 'pk5'}
2 {'pk2'}
-----
useless
0 {'pk4', 'pk7', 'pk6', 'pk3', 'pk1', 'pk5', 'pk2'}
-----


In [24]:
ddb.upsert_meta('pk4', {'vp': True})
for field, index in ddb.indexes.items():
    if field == 'vp':
        for k in index.keys():
            print(k, index[k])

False {'pk7', 'pk6', 'pk3', 'pk1', 'pk5', 'pk2'}
True {'pk4'}


In [25]:
ddb.upsert_meta('pk5', {'vp': True})
for field, index in ddb.indexes.items():
    if field == 'vp':
        for k in index.keys():
            print(k, index[k])

False {'pk3', 'pk6', 'pk7', 'pk1', 'pk2'}
True {'pk4', 'pk5'}


In [26]:
ddb.upsert_meta('pk5', {'vp': False})
for field, index in ddb.indexes.items():
    if field == 'vp':
        for k in index.keys():
            print(k, index[k])

False {'pk7', 'pk6', 'pk3', 'pk1', 'pk5', 'pk2'}
True {'pk4'}


In [27]:
ddb.upsert_meta('pk5', {'vp': True})
for field, index in ddb.indexes.items():
    if field == 'vp':
        for k in index.keys():
            print(k, index[k])

False {'pk3', 'pk6', 'pk7', 'pk1', 'pk2'}
True {'pk4', 'pk5'}


In [28]:
ddb.delete_ts('pk5')

In [29]:
for field, index in ddb.indexes.items():
    if field == 'vp':
        for k in index.keys():
            print(k, index[k])

False {'pk3', 'pk6', 'pk7', 'pk1', 'pk2'}
True {'pk4'}


In [30]:
ddb.delete_ts('pk4')

In [31]:
for field, index in ddb.indexes.items():
    if field == 'vp':
        for k in index.keys():
            print(k, index[k])

False {'pk3', 'pk6', 'pk7', 'pk1', 'pk2'}


In [32]:
ddb.insert_ts('pk4', a1)
ddb.insert_ts('pk5', a1)
ddb.upsert_meta('pk4', {'vp': True})
ddb.upsert_meta('pk5', {'vp': True})

In [33]:
ddb.select({'vp': True}, ['vp'], None)

(['pk4', 'pk5'], [{'vp': True}, {'vp': True}])

In [34]:
ddb.select({'vp': False}, ['vp'], None)

(['pk3', 'pk1', 'pk7', 'pk6', 'pk2'],
 [{'vp': False}, {'vp': False}, {'vp': False}, {'vp': False}, {'vp': False}])

In [35]:
# bitmap index errors??
print(ddb.indexes['vp'].values())
print(ddb.indexes['vp'].items())

[{'pk3', 'pk6', 'pk7', 'pk1', 'pk2'}, {'pk4', 'pk5'}]
[(False, {'pk3', 'pk6', 'pk7', 'pk1', 'pk2'}), (True, {'pk4', 'pk5'})]


In [36]:
[ddb.indexes['vp'][k] for k in ddb.indexes['vp'].keys()]

[{'pk1', 'pk2', 'pk3', 'pk6', 'pk7'}, {'pk4', 'pk5'}]

In [37]:
print(ddb.indexes['vp'][True])
print(ddb.indexes['vp'][False])

{'pk4', 'pk5'}
{'pk3', 'pk6', 'pk7', 'pk1', 'pk2'}


In [39]:
# print metadata
for field, index in ddb.indexes.items():
    print(field, index.items())

std [(0.0, {'pk4', 'pk7', 'pk6', 'pk3', 'pk1', 'pk5', 'pk2'})]
blarg [(0, {'pk4', 'pk7', 'pk6', 'pk3', 'pk1', 'pk5'}), (2, {'pk2'})]
vp [(False, {'pk3', 'pk6', 'pk7', 'pk1', 'pk2'}), (True, {'pk4', 'pk5'})]
deleted [(False, {'pk4', 'pk7', 'pk6', 'pk3', 'pk1', 'pk5', 'pk2'})]
mean [(0.0, {'pk4', 'pk7', 'pk6', 'pk3', 'pk1', 'pk5', 'pk2'})]
order [(0, {'pk4', 'pk7', 'pk6', 'pk3', 'pk1', 'pk5'}), (2, {'pk2'})]
useless [(0, {'pk4', 'pk7', 'pk6', 'pk3', 'pk1', 'pk5', 'pk2'})]


In [40]:
ddb = None

In [41]:
# reload
ddb = PersistentDB(schema, 'pk', len(t), data_dir='../db_files')

In [42]:
# print metadata
for field, index in ddb.indexes.items():
    print(field, index.items())

std [(0.0, {'pk4', 'pk7', 'pk6', 'pk3', 'pk1', 'pk5', 'pk2'})]
blarg [(0, {'pk4', 'pk7', 'pk6', 'pk3', 'pk1', 'pk5'}), (2, {'pk2'})]
vp [(False, {'pk3', 'pk6', 'pk7', 'pk1', 'pk2'}), (True, {'pk4', 'pk5'})]
deleted [(False, {'pk4', 'pk7', 'pk6', 'pk3', 'pk1', 'pk5', 'pk2'})]
mean [(0.0, {'pk4', 'pk7', 'pk6', 'pk3', 'pk1', 'pk5', 'pk2'})]
order [(0, {'pk4', 'pk7', 'pk6', 'pk3', 'pk1', 'pk5'}), (2, {'pk2'})]
useless [(0, {'pk4', 'pk7', 'pk6', 'pk3', 'pk1', 'pk5', 'pk2'})]
